In [ ]:
import time
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from scipy.cluster.vq import kmeans2
import pandas as pd

##Coarse Quantizer
The coarse quantizer is for non-exhaustive search. It retrieves a candidate set first, then searches within the candidate set for nearest neighbors based on PQ.

split the space into k clusters of vectors
• usually clusters are defined by k-means
• assign vectors to nearest centroid
• index = inverted list structure
• maps centroid id -> list of vectors assigned to it

Coarse-quantization: Given a query vector y ∈
RD, the nearest bucket Xj is selected. The residual
y−μj between the query and the representative vector
of the jth bucket is computed.


K coarse centers ¯C = {¯ck }K
k=1
are created by running the
clustering algorithm [35] on ¯X (or its subset). Note that each coarse
center is also a PQ-code ¯ck ∈ {1, . . . ,Z}M. Using these coarse
centers, the database PQ-codes ¯X are clustered into K groups. The
resulting assignments are stored as posting listsW = {Wk }K
k=1
,
where eachWk is a set of identifiers of the database vectors whose
nearest coarse center is the kth one:

In [ ]:
class KmeansCoarseQuantizer:

    def __init__(self, dimension, n_list):
        self.dimension = dimension
        self.n_list = n_list
        self.coarse_quantizer = KMeans(n_clusters=n_list)

    def fit(self, train):
        model = self.coarse_quantizer.fit(train)
        self.centroids = model.cluster_centers_
        self.model = model
        self.labels = model.predict(train)

    def add(self, vecs):
      self.labels = self.model.predict(vecs)

    def get_labels(self):
        return self.labels

    def get_centroids(self):
        return self.centroids

In [ ]:
class IVFADC:

    def __init__(self, inverted_file_list, dimensions, number_subvectors, n_bits):
          self.inverted_file_list = inverted_file_list
          self.number_subvectors = number_subvectors
          self.number_subspaces = 2 ** n_bits
          self.d = dimensions

    def train(self, train_vectors, sub_size='all'):
      #sub_size specify the number of random element chosen from the entire training set, or the indexes to perform the coarse quantizaion
        start_time = time.time()
        
        if sub_size == 'all':
          self.inverted_file_list.fit(train_vectors)
        elif type(sub_size) is np.ndarray:
          self.inverted_file_list.fit(train_vectors[sub_size])
        else:
          train_subset_index = np.random.choice(train_vectors.shape[0], sub_size, replace=False)
          self.inverted_file_list.fit(train_vectors[train_subset_index])
        
        self.inverted_file_list.add(train_vectors)

        codebooks = []
        index = []
        for i in range(len(self.inverted_file_list.get_centroids())):
            points_id = np.where(self.inverted_file_list.get_labels() == i)[0]
            residuals = np.subtract(train_vectors[points_id], self.inverted_file_list.get_centroids()[i])  # compute the residuals
            codewords, codebook = self.productQuantization(residuals)
            codebooks.append(codebook)
            index.append([points_id, codewords])

        train_time = time.time() - start_time
        self.train_time = train_time
        self.codebooks = codebooks
        self.index = index
    
    def get_train_time(self):
      return self.train_time

    def productQuantization(self, vecs):
        M = self.number_subvectors
        Ks = self.number_subspaces
        assert vecs.ndim == 2
        N, D = vecs.shape
        Ds = int(D / M)

        codebook = np.zeros((Ks, D))
        subquantizers = np.zeros((N, M), dtype=np.uint8)  # 8 bit

        for i, vecs_sub in enumerate(np.split(vecs, M, axis=1)):
            centroids, labels = kmeans2(vecs_sub, Ks, minit='points')
            codebook[:, i * Ds: (i + 1) * Ds] = centroids
            subquantizers[:, i] = labels
        return subquantizers, codebook

    def search(self, q, n_neighbours, n_probe, metric):

        assert self.d % self.number_subvectors == 0
        start = time.time()

        dim_subvector = int(self.d / self.number_subvectors)

        self.n_probe = n_probe

        distances_matrix_queries = cdist(q, self.inverted_file_list.get_centroids(), metric='sqeuclidean')  # compute multidimensional distances
        clusters_ID = distances_matrix_queries.argsort(axis=1)[:, :n_probe]

        # Compute nearest images from queries
        neighbours = []
        neighbours_distances = []

        for i, query in enumerate(clusters_ID):
            Similarities_q = []  # keep all the ... between the query and the kNN in each accessed cluster
            point_ID = []
            for centroid_id in query:
                entry = self.index[centroid_id]
                query_residual = np.subtract(q[i], self.inverted_file_list.get_centroids()[centroid_id])

                # summing up the contribution distance of each sub_vector
                query_sim = np.zeros((1, entry[1].shape[0]))
                for m, vecs_sub in enumerate(np.split(entry[1], self.number_subvectors, axis=1)):
                    sub_query = query_residual[m * dim_subvector:(m + 1) * dim_subvector].reshape(1, -1)
                    # reconstruct the apprisimated vector using centroids' ID: codebooks[centroid_id][vecs_sub.ravel(), m * dim_subvector:(m + 1) * dim_subvector]
                    if metric == 'sqeuclidean':
                      sim_query_sub_centroids = cdist(sub_query,
                                                      self.codebooks[centroid_id][:, m * dim_subvector:(m + 1) * dim_subvector],
                                                      metric='sqeuclidean')
                      
                    elif metric == 'cosine':
                      sim_query_sub_centroids = cdist(sub_query, 
                                                      self.codebooks[centroid_id][:, m * dim_subvector:(m + 1) * dim_subvector], 
                                                      metric='cosine')
                      sim_query_sub_centroids = np.subtract(np.ones(sim_query_sub_centroids.shape), sim_query_sub_centroids)

                    elif metric == 'dot':
                      sim_query_sub_centroids = sub_query.dot(self.codebooks[centroid_id][:, m * dim_subvector:(m + 1) * dim_subvector].T)
                      
                    else:
                      print("Metric not allowed!")
                      exit()
                    
                    sim_contributions = sim_query_sub_centroids[:, vecs_sub.ravel()]
                    query_sim = np.sum([sim_contributions, query_sim], axis=0)
                
                Similarities_q += query_sim[0].tolist()
                point_ID += entry[0].tolist()

            if metric == 'sqeuclidean':
              indexes = np.argsort(np.array(Similarities_q))[:n_neighbours]
            elif metric == 'cosine' or metric == 'dot':
              indexes = np.argsort(np.array(Similarities_q))[::-1][:n_neighbours]

            neighbours.append(np.array(point_ID)[indexes])
            neighbours_distances.append(np.array(Similarities_q)[indexes])

            search_time = time.time() - start
            dist_computed = n_probe*self.number_subspaces
        return np.array(neighbours), np.array(neighbours_distances), search_time

    def compute_recall(self, true_neighbors, predicted_neighbors):
        recalls = []
        for t, p in zip(true_neighbors, predicted_neighbors):
            intersection = np.intersect1d(t, p)  # find common elements
            recall = len(intersection) / len(t)
            recalls.append(recall)

        return np.mean(recalls)

    def save_index(self, path_index):
      with open(path_index, 'wb') as f:
        pickle.dump(self, f)
      return